In [1]:
import os

In [2]:
%pwd

'd:\\DS\\Projects\\DL\\Workplace-Safety-Detection-CV\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'd:\\DS\\Projects\\DL\\Workplace-Safety-Detection-CV'

In [5]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path
    dataset_name: str
    data_yaml_file: Path

In [6]:
import os
from CV.constants import *
from CV.utils.common import read_yaml, create_directories
from CV.entity.config_entity import DataIngestionConfig

class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])


    
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir, 
            dataset_name=config.dataset_name,
            data_yaml_file=config.data_yaml_file
        )
        return data_ingestion_config

In [7]:
import os
import zipfile
import yaml
import gdown
from CV import logger
from CV.entity.config_entity import DataIngestionConfig
from pathlib import Path

class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config


    
     
    def download_file(self)-> str:
        '''
        Fetch data from the url
        '''

        try: 
            dataset_url = self.config.source_URL
            zip_download_dir = self.config.local_data_file
            os.makedirs("artifacts/data_ingestion", exist_ok=True)
            logger.info(f"Downloading data from {dataset_url} into file {zip_download_dir}")

            file_id = dataset_url.split("/")[-2]
            prefix = 'https://drive.google.com/uc?/export=download&id='
            gdown.download(prefix+file_id,zip_download_dir)

            logger.info(f"Downloaded data from {dataset_url} into file {zip_download_dir}")

        except Exception as e:
            raise e
        
    
    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)




    def update_data_yaml_file(self):
        """
        Modify paths inside a data.yaml file.

        Returns:
            None: The function modifies the file in-place.
        """

        # Path to the data.yaml file
        data_yaml_path = Path("artifacts/data_ingestion/Workplace-Safety-Dataset/data.yaml")
        new_paths={
            "train": "artifacts/data_ingestion/Workplace-Safety-Dataset/train/images",
            "val": "artifacts/data_ingestion/Workplace-Safety-Dataset/valid/images",
            "test": "artifacts/data_ingestion/Workplace-Safety-Dataset/test/images"
            }

        # Load the data.yaml file
        if not data_yaml_path.is_file():
            raise FileNotFoundError(f"data.yaml file not found at {data_yaml_path}")



        with open(data_yaml_path, "r") as f:
            data_yaml = yaml.safe_load(f)


        # Modify the paths
        for key, new_path in new_paths.items():
            try:
                data_yaml[key] = new_path
            except Exception as e:
                raise e
    
        # Save the updated data.yaml
        with open(data_yaml_path, "w") as f:
            yaml.dump(data_yaml, f)
 
        with open(data_yaml_path, "r") as f:
            data_yaml = yaml.safe_load(f)
    
        # Resolve the paths for train and val
        train_path = Path(data_yaml["train"]).resolve()
        val_path = Path(data_yaml["val"]).resolve()
        test_path = Path(data_yaml["test"]).resolve()

        # Check if the directories exist
        if not train_path.is_dir():
            raise FileNotFoundError(f"Training directory not found: {train_path}")

        if not val_path.is_dir():
            raise FileNotFoundError(f"Validation directory not found: {val_path}")

        # Update the paths in the data.yaml dictionary
        data_yaml["train"] = str(train_path)
        data_yaml["val"] = str(val_path)
        data_yaml["test"] = str(test_path)

        # Save the updated data.yaml
        with open(data_yaml_path, "w") as f:
            yaml.dump(data_yaml, f)

In [8]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
    data_ingestion.update_data_yaml_file()
except Exception as e:
    raise e

[2025-01-22 21:03:59,266: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-01-22 21:03:59,273: INFO: common: yaml file: params.yaml loaded successfully]
[2025-01-22 21:03:59,275: INFO: common: created directory at: artifacts]
[2025-01-22 21:03:59,277: INFO: common: created directory at: artifacts/data_ingestion]
[2025-01-22 21:03:59,279: INFO: 615811023: Downloading data from https://drive.google.com/file/d/1297WLL9_ULuZtsnYN_SOseNEiEbv15Dd/view?usp=sharing into file artifacts/data_ingestion/data.zip]


Downloading...
From (original): https://drive.google.com/uc?/export=download&id=1297WLL9_ULuZtsnYN_SOseNEiEbv15Dd
From (redirected): https://drive.google.com/uc?%2Fexport=download&id=1297WLL9_ULuZtsnYN_SOseNEiEbv15Dd&confirm=t&uuid=0fcc9e75-48fa-44b8-9d70-766506deeb44
To: d:\DS\Projects\DL\Workplace-Safety-Detection-CV\artifacts\data_ingestion\data.zip
100%|██████████| 102M/102M [00:24<00:00, 4.13MB/s] 

[2025-01-22 21:04:28,528: INFO: 615811023: Downloaded data from https://drive.google.com/file/d/1297WLL9_ULuZtsnYN_SOseNEiEbv15Dd/view?usp=sharing into file artifacts/data_ingestion/data.zip]
